# Cars dataset exploration with plotly.py version 3.0

In [ ]:
# Apply hack to hide tracebacks for presentation
import hide_ipython_tbs

## Load cars dataset

In [ ]:
import pandas as pd
import numpy as np

cars_df = pd.read_csv('data/cars/cars.csv',
                      usecols=['City mpg',
                               'Fuel Type',
                               'Horsepower',
                               'Model Year',
                               'Torque', 'Hybrid', 'ID'])
cars_df.sample(5)

In [ ]:
cars_df.shape

## Load images of cars

In [ ]:
import os

image_data = {}
for img_filename in os.listdir('data/cars/images'):
    model_year = img_filename.split('.')[0]
    with open(f"data/cars/images/{img_filename}", "rb") as f:
        b = f.read()
        image_data[model_year] = b

In [ ]:
from ipywidgets import Image
Image(value=image_data['2012_Chevrolet_Camaro_Coupe'])

## Construct plotly.py Figure Widget
Torqe vs. MPG Scatter Trace

In [ ]:
import plotly.graph_objs as go

In [ ]:
fig = go.FigureWidget(
    data=[
        dict(
            type='scattergl',
            x=cars_df['Torque'],
            y=cars_df['City mpg'],
            mode='markers',
        )
    ],
)

### Display Figure
Before online or offline `iplot`. Still supported, but not needed with `FigureWidget`

In [ ]:
fig

### Label Figure
Use property assignment syntax to:

Set `fig.layout.title` to `'Torque and Fuel Efficience'`

In [ ]:
fig.layout.title = 'Torque and Fuel Efficience'

Check default font size

In [ ]:
fig.layout.titlefont.size

Increase the title font size

In [ ]:
fig.layout.titlefont.size = 22

Set `fig.layout.titlefont.family` to `'Rockwell'`

In [ ]:
fig.layout.titlefont.family = 'Rockwell'

### Create New View for Figure
If working in JupyterLab, right-click on blue bar to the left of the figure and select "Create New View for Output". Drag view to the right half of the screen.

### Label Axes

Set the `fig.layout.xaxis.title` property to `'Torque (foot-pounds)'`

In [ ]:
fig.layout.xaxis.title = 'Torque (foot-pounds)'

Set the `fig.layout.yaxis.title` property to `'City MPG'`

In [ ]:
fig.layout.yaxis.title = 'City MPG'

### Notice Quantization
Zoom in and notice that the dataset is quantized

### Apply Jitter

In [ ]:
scatter = fig.data[0]
scatter

In [ ]:
N = len(cars_df)
scatter.x = scatter.x + np.random.rand(N) * 10
scatter.y = scatter.y + np.random.rand(N) * 1

Zoom level did not reset! Plot is updated in-place. Not recreated each time a property changes

### Address Overplotting

Lower marker opacity

In [ ]:
scatter.marker.opacity = 0.2

Decrease marker size

In [ ]:
scatter.marker.size = 4

### Aside on validation

What if I though opacity ranged from 0 to 255?

In [ ]:
scatter.marker.opacity = 50

What if I forgot the name of an enumeration value?

In [ ]:
fig.layout.hovermode = 'nearest' # Set to 'closest'

What if I don't know how to spell 'fuchsia'?

In [ ]:
scatter.marker.color = 'fushia' # Set to 'fuchsia'

Restore default marker color

In [ ]:
scatter.marker.color = None

### Add density contour

Add smoothed density contour trace (`histogram2dcontour`) based on `scatter.x` and `y=scatter.y` values.

In [ ]:
contour = fig.add_histogram2dcontour(
    x=scatter.x, y=scatter.y)

Set contour colorscale

In [ ]:
contour.colorscale = 'Hot'

Reverse the colorscale

In [ ]:
contour.reversescale = True

Disable tooltips for contour

In [ ]:
contour.hoverinfo = 'skip'

Tweak marker size and opacity

In [ ]:
scatter.marker.opacity = .1
scatter.marker.size = 3

### Create marker configuration widget

Define function that inputs `opacity` and `size` and updates the figure.

In [ ]:
def set_opacity(opacity, size):
    scatter.marker.opacity = opacity
    scatter.marker.size = size

Use `ipywidgets.interactive` to generate control panel for function.

In [ ]:
from ipywidgets import interactive
opacity_slider = interactive(set_opacity,
                             opacity=(0.0, 1.0, 0.01),
                             size=(1, 10, 0.25))
opacity_slider

Adjust the width of the slider widgets

In [ ]:
opacity_slider.children[0].layout.width = '400px'
opacity_slider.children[1].layout.width = '400px'

Try zooming and then adjusting the marker params

### Looking at outliers

#### Tooltips
Use `'ID'` column as tooltip for scatter

In [ ]:
scatter.text = cars_df['ID']
scatter.hoverinfo = 'text'

#### All properties

Create an HTML widget to display the hover properties

In [ ]:
from ipywidgets import HTML
details = HTML()
details

Register callback function to be executed on hover events. It will update the HTML widget using the pandas `to_html` method.

In [ ]:
def hover_fn(trace, points, state):
    ind = points.point_inds[0]
    details.value = cars_df.iloc[ind].to_frame().to_html()

scatter.on_hover(hover_fn)

#### Vehicle image

Create an `ipywidgets.Image` widget to display images

In [ ]:
from ipywidgets import Image, Layout
image_widget = Image(
    value=image_data['2012_Chevrolet_Camaro_Coupe'],
    layout=Layout(height='252px', width='400px')
)
image_widget

Update hover function to update the image widget along with the HTML widget

In [ ]:
def hover_fn(trace, points, state):

    ind = points.point_inds[0]
    
    # Update details HTML widget
    details.value = cars_df.iloc[ind].to_frame().to_html()
    
    # Update image widget
    model_year = cars_df['Model Year'][ind].replace(' ', '_')
    image_widget.value = image_data[model_year]

scatter.on_hover(hover_fn)

## Bringing it all together

Create simple dashboard using `HBox` and `VBox` containers

In [ ]:
from ipywidgets import HBox, VBox
VBox([fig,
      opacity_slider,
      HBox([image_widget, details])])